# Анализ базы данных с помощью SQL

**Цель исследования** - проанализировать базу данных сервиса для чтения книг по подписке:

- Собрать информацию по книгам (количество книг, средняя оценка, количество обзоров);
- Обозначить самое крупное издательство по выпущенным книгам;
- Обозначить самых популярных авторов;
- Найти среднее количетсво отзывов активных пользователей;
- Дать общие рекомендации.

**Основные шаги:**

- [1. Изучение общей информации из файлов.](#step1) 
- [2. Работа с базой данных.](#step2)
- [3. Выводы и рекомендации.](#step3)

## 1. Изучение общей информации из файлов. <a id="step1"></a>    

### 1.1. Получение данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
     'pwd': 'Sdf4$2;d-d30pp', # пароль
     'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
     'port': 6432, # порт подключения
     'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                     db_config['pwd'],
                     db_config['host'],
                     db_config['port'],
                     db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [3]:
pd.read_sql('select * from books', con = engine).head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
pd.read_sql('select * from books', con = engine).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
book_id             1000 non-null int64
author_id           1000 non-null int64
title               1000 non-null object
num_pages           1000 non-null int64
publication_date    1000 non-null object
publisher_id        1000 non-null int64
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

In [5]:
pd.read_sql('select * from authors', con = engine).head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [6]:
pd.read_sql('select * from authors', con = engine).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
author_id    636 non-null int64
author       636 non-null object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

In [7]:
pd.read_sql('select * from publishers', con = engine).head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [8]:
pd.read_sql('select * from publishers', con = engine).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
publisher_id    340 non-null int64
publisher       340 non-null object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства.

In [9]:
pd.read_sql('select * from ratings', con = engine).head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [10]:
pd.read_sql('select * from ratings', con = engine).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
rating_id    6456 non-null int64
book_id      6456 non-null int64
username     6456 non-null object
rating       6456 non-null int64
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

In [11]:
pd.read_sql('select * from reviews', con = engine).head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [12]:
pd.read_sql('select * from reviews', con = engine).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
review_id    2793 non-null int64
book_id      2793 non-null int64
username     2793 non-null object
text         2793 non-null object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

### 1.1. Выводы

В базе данных содержится:
- 1000 книг;
- 636 авторов;
- 6456 оценок;
- 2793 обзоров;
- 340 издательств.

## 2. Работа с базой данных. <a id="step2"></a>    

### 2.1. Определение количества книг, вышедших после 1 января 2000 года

In [13]:
query = '''
    SELECT
        COUNT(DISTINCT book_id) AS number_of_books
    FROM
        books
    WHERE
        publication_date >  '1999-12-31';
'''
pd.io.sql.read_sql(query, con = engine)

,number_of_books
0,821


В базе данных хранится 821 книга, вышедшая после 1 января 2000 года.

### 2.2. Количество обзоров и средняя оценка книг

In [19]:
query = '''
    SELECT
        books.book_id,
        books.title,
        COUNT(DISTINCT reviews.review_id) AS count_review,
        AVG(ratings.rating) AS avg_rating
    FROM
        books
        LEFT JOIN reviews ON reviews.book_id = books.book_id
        LEFT JOIN ratings ON ratings.book_id = books.book_id
    GROUP BY
        books.book_id
    ORDER BY
        count_review DESC,
        avg_rating DESC
    LIMIT
        10;
'''
pd.io.sql.read_sql(query, con = engine)

,book_id,title,count_review,avg_rating
0,948,Twilight (Twilight #1),7,3.662500
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,656,The Book Thief,6,4.264151
4,734,The Glass Castle,6,4.206897
5,497,Outlander (Outlander #1),6,4.125000
6,750,The Hobbit or There and Back Again,6,4.125000
7,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
8,779,The Lightning Thief (Percy Jackson and the Oly...,6,4.080645
9,963,Water for Elephants,6,3.977273


В лидерах по количеству обзоров находятся книги из популярных серий и бестселлеры разных годов.   

### 2.3. Издательство, которое выпустило наибольшее число книг

In [15]:
query = '''
    SELECT
        publishers.publisher_id,
        publishers.publisher,
        COUNT(DISTINCT books.book_id) AS count_books
    FROM
        books
        LEFT JOIN publishers ON publishers.publisher_id = books.publisher_id
    WHERE 
        books.num_pages > 50
    GROUP BY
        publishers.publisher_id
    ORDER BY
        count_books DESC
    LIMIT
        5;      
'''
pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher,count_books
0,212,Penguin Books,42
1,309,Vintage,31
2,116,Grand Central Publishing,25
3,217,Penguin Classics,24
4,33,Ballantine Books,19


Британское издательство **Penguin Books** выпустило наибольшее количество книг.  

### 2.4.  Автор с самой высокой средней оценкой книг 

In [16]:
query = '''
    SELECT 
        SUBQ2.author,
        AVG(SUBQ1.avg_rating) AS avg_rating,
        SUM(SUBQ1.cnt_rating) AS cnt_rating
    FROM
        (SELECT
            books.book_id,
            books.author_id,
            AVG(ratings.rating) AS avg_rating,
            COUNT(ratings.rating) AS cnt_rating
        FROM
            books
        LEFT JOIN
            ratings ON books.book_id = ratings.book_id    
        GROUP BY
            books.book_id   
        HAVING
            COUNT(ratings.rating)>= 50) AS SUBQ1
            
    LEFT JOIN
        (SELECT
            authors.author,
            authors.author_id
        FROM
            authors
        LEFT JOIN
            books ON authors.author_id = books.author_id
        GROUP BY
            authors.author,
            authors.author_id
            ) AS SUBQ2 on SUBQ1.author_id = SUBQ2.author_id
    GROUP BY
        SUBQ2.author
    ORDER BY
        AVG(SUBQ1.avg_rating) DESC;
'''
pd.io.sql.read_sql(query, con = engine)  

,author,avg_rating,cnt_rating
0,J.K. Rowling/Mary GrandPré,4.283844,310.0
1,Markus Zusak/Cao Xuân Việt Khương,4.264151,53.0
2,J.R.R. Tolkien,4.258446,162.0
3,Louisa May Alcott,4.192308,52.0
4,Rick Riordan,4.080645,62.0
5,William Golding,3.901408,71.0
6,J.D. Salinger,3.825581,86.0
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474,57.0
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879,66.0
9,Dan Brown,3.754540,143.0


На первом месте - **Джоан Роулинг**, на втором - **Маркус Зусак**, на третьем - **Джон Рональд Руэл Толкин**.

### 2.5.  Среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [17]:
query = '''
    SELECT
        ROUND(AVG(SUBQ3.cnt_reviews),2) as "Среднее количество обзоров"
    FROM
        (SELECT
            SUBQ1.username,
            SUBQ1.cnt_reviews,
            SUBQ2.cnt_rating 
        FROM
            (SELECT   
                reviews.username,
                COUNT(reviews.review_id) AS cnt_reviews
            FROM
                reviews
            INNER JOIN books ON reviews.book_id = books.book_id        
            GROUP BY
                reviews.username
            ORDER BY
                cnt_reviews DESC) AS SUBQ1
        
    INNER JOIN
        (SELECT
            ratings.username,
            COUNT(ratings.rating) AS cnt_rating  
         FROM
            ratings
         INNER JOIN books ON ratings.book_id = books.book_id
         GROUP BY
            ratings.username
         HAVING
            COUNT(ratings.rating) > 50) AS SUBQ2 ON SUBQ2.username = SUBQ1.username   
        ORDER BY
        SUBQ1.cnt_reviews DESC) AS SUBQ3

'''
pd.io.sql.read_sql(query, con = engine)

,Среднее количество обзоров
0,24.33


Активный пользователь в среднем пишет 24 обзора.

## 3. Выводы и рекомендации. <a id="step3"></a>    

Выводы:
- С 01 января 2000 года выпущена **821 книга**;
- Самая обозреваемая книга - **Сумерки - 7 обзоров**;
- Самое активное издательство - **холдинг Penguins**;
- Самый высокая оценка у книг **Джоан Роулинг**;
- В среднем наиболее активные пользователи пишут **24 отзыва**.

Рекомендации:

- При выборе ассортимента книг следует сделать ставку на популярные серии книг и бестселлеры;
- Важна обратная связь от пользователей, возможность оценивать книги и писать отзывы.